<center>
  <div style="font-size: 26px">Сбор Данных об организации</div>
</center>

***
Собираем данные:
 * Id
 * Название
 * Координаты
 * Кол-во отзывов
 * Рейтинг

***
Для работы нам понадобятся **библиотеки**:
 * Selenium
 * Pandas
 * bs4
 * IPython

**Установка через pip:** `pip install selenium Pandas bs4 IPython`

**Установка через conda:**`conda install -c conda-forge selenium Pandas IPython`

***
**Импортирование необходимых библиотек**

In [1]:
from time import sleep
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import clear_output # необходимо для удобного вывода информации

import json


***
Необходимо ввести 3 переменные:
 * Населенный пункт
 * Заведение
 * Объёмом выборки


In [2]:
location = 'Москва' # населенный пункт
title = 'тц' # заведение
count_of_units = 1000 # объем выборки

***
Настраиваем работу драйвера

In [3]:
driver = webdriver.Chrome()
# Открыть URL
url = f'https://yandex.ru/maps/213/moscow/search/{location}%20{title}'
driver.get(url)

In [13]:
# Ожидание элемента до тех пор, пока он не будет найден
wait = WebDriverWait(driver, 100)  # 10 секунд ожидания

elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "search-snippet-view")))

len(elements) #Смотрим кол-во собранных заведений

965

***
Сбор данных в блоках в различные списки

In [14]:
id_list = []
coord_list = []
name_list = []
ratings_count_list = []
rating_list = []
for elem in elements:
    search_body = elem.find_element(By.CLASS_NAME, "search-snippet-view__body")

    id = search_body.get_attribute('data-id')
    id_list.append(id)

    coord = search_body.get_attribute('data-coordinates')
    coord_list.append(coord)

    try:
        search_name = elem.find_element(By.CLASS_NAME, 'search-business-snippet-view__title')
        name = search_name.text
        name_list.append(name)
    except:
        name_list.append(None)

    try:
        search_count = elem.find_element(By.CLASS_NAME, 'business-rating-with-text-view__count')
        count = search_count.text.split()[0]
        count = count.replace('(', '')
        count = count.replace(')', '')
        
        ratings_count_list.append(count)
    except:
        ratings_count_list.append(None)

    try:
        search_rating = elem.find_element(By.CLASS_NAME, 'business-rating-badge-view__rating-text')
        rating = search_rating.text
        rating_list.append(rating)
    except:
        rating_list.append(None)

***
Сбор необходимых данных в JSON формат

In [17]:
unit = {"id": "1234", "name": "Aboba", "coordinates": "37,25", "reviews_count": "44", "rating": "5.0"}

name = location.split()[0]

json_file = open(name + "_output.json", "w", encoding="utf-8")

json_file.write('[')

for i in range(0, len(id_list)):
    if id_list[i] == None:
        continue
    unit["id"] = id_list[i]
    unit["coordinates"] = coord_list[i]
    unit["name"] = name_list[i]
    unit["reviews_count"] = ratings_count_list[i]
    unit["rating"] = rating_list[i]

    json.dump(unit, json_file, ensure_ascii=False, indent=10)
    json_file.write(',\n')

json_file.write(']')

1